In [1]:
import os
import torch
import torch.nn as nn
import numpy as np
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data import WeightedRandomSampler
from torchvision.transforms import v2
import utils

In [2]:
BATCH_SIZE = 10
SHUFFLE = False

# Train augmentations
transformation = v2.Compose([
    v2.RandomHorizontalFlip(p=0.5),
    v2.RandomVerticalFlip(p=0.5),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=utils.MEANS, std=utils.STDS),
    transforms.ToTensor()
])
# Validation augmentation
transformation_val = v2.Compose([
    v2.Normalize(mean=utils.MEANS, std=utils.STDS),
    transforms.ToTensor()
])

train_data = utils.CustomImageDataset(label_dir=os.path.join(utils.BASE_PATH_DATA, utils.LABEL_PATH_TRAIN), img_dir=os.path.join(utils.BASE_PATH_DATA, utils.IMG_PATHS_TRAIN), transform=transformation, use_selected_bands=True)
labels = train_data.img_labels
class_sample_count = np.array(
    [len(np.where(labels == t)[0]) for t in np.unique(labels)])

weight = 1. / class_sample_count
samples_weight = np.array([weight[t] for t in labels])

samples_weight = torch.from_numpy(samples_weight)
sampler = WeightedRandomSampler(samples_weight.type('torch.DoubleTensor'), len(samples_weight))

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=SHUFFLE, sampler=sampler)

val_data = utils.CustomImageDataset(os.path.join(utils.BASE_PATH_DATA, utils.LABEL_PATH_VAL), os.path.join(utils.BASE_PATH_DATA, utils.IMG_PATHS_VAL), transform=transformation_val, use_selected_bands=True)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=SHUFFLE)

test_data = utils.CustomImageDataset(os.path.join(utils.BASE_PATH_DATA, utils.LABEL_PATH_TEST), os.path.join(utils.BASE_PATH_DATA, utils.IMG_PATHS_TEST), transform=transformation_val)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=SHUFFLE)

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(6, 6, 5) # # 12 bands = 12 input channels
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.dropout = nn.Dropout(0.3)
        self.fc1 = nn.Linear(256, 120)
        self.fc2 = nn.Linear(120, 2)
        #self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Implement the forward function in the network
        x = self.conv1(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = x.flatten(1)
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.fc2(x)
        #x = self.sigmoid(x)
        return x


In [4]:
# Hyperparams. Set these to reasonable values
# TODO
LEARNING_RATE = 0.0001
EPOCHS = 45

# Load our network
model = Net()

# Define our loss function
criterion = nn.CrossEntropyLoss()

# Define our optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# Train the model
utils.train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=EPOCHS, cnn=True, sigmoid=False, show_log=True, show_plot=True)

Epoch 0, training loss: 0.06851912324245159, validation loss: 0.08028753019041485
Train accuracy = 0.55
Validation accuracy = 0.2777777777777778
Epoch 1, training loss: 0.06792412170997032, validation loss: 0.08022736509641011
Train accuracy = 0.5076923076923077
Validation accuracy = 0.2777777777777778
Epoch 2, training loss: 0.0654497566131445, validation loss: 0.07832766655418608
Train accuracy = 0.5384615384615384
Validation accuracy = 0.3055555555555556
Epoch 3, training loss: 0.06178480375271577, validation loss: 0.07827796621455087
Train accuracy = 0.5923076923076923
Validation accuracy = 0.4027777777777778
Epoch 4, training loss: 0.06156411904555101, validation loss: 0.06425513244337505
Train accuracy = 0.6807692307692308
Validation accuracy = 0.8055555555555556
Epoch 5, training loss: 0.056656916668781865, validation loss: 0.06268290927012761
Train accuracy = 0.7307692307692307
Validation accuracy = 0.7777777777777778
Epoch 6, training loss: 0.051474300714639515, validation los

: 

## Results
1) Loss: CrossEntropy; Kernel Size: 5; Optimization: Adam; Learning rate: 0.0001, Epochs: 30; all bands \
lowest loss for validation set: 0.03579433603833119, with an accuracy of 0.8888888888888888
2) Loss: CrossEntropy; Kernel Size: 3; Optimization: Adam; Learning rate: 0.0001, Epochs: 30; all bands \
lowest loss for validation set: 0.030835631613930065, with an accuracy of 0.8888888888888888
3) Loss: CrossEntropy; Kernel Size: 5; Optimization: Adam; Learning rate: 0.0001, Epochs: 30; all bands; with our normalization \
lowest loss for validation set: 0.040424768502513565, with an accuracy of 0.8888888888888888
4) Loss: CrossEntropy; Kernel Size: 5; Optimization: Adam; Learning rate: 0.0001, Epochs: 30; all bands; with our normalization, with Sigmoid \
lowest loss for validation set: 0.5518322504229016, with an accuracy of 0.8472222222222222
5) 


To test:
- different bands
- learning rate
- Sigmoid
- loss






### Results
| Parameter   | Value       | Value       | Value       |
| ----------- | ----------- | ----------- | ----------- |
| Loss Func   | CrossEntropy | CrossEntropy | CrossEntropy |
| Kernel Size | 5 | 5 | 5 |
| Optimization | Adam | Adam | Adam |
| Learning Rate | 0.0001 | 0.0001 | 0.0001 |
| Epochs | 30 | 30 | 50 |
| Normalization | yes | yes | yes |
| Bands used | 12 | 12 | 12 |
| Layers used | no Sigmoid | no Sigmoid | no Sigmoid |
| Augmentation | yes | yes | yes |
| Dropout | no | no | no |
| **lowest loss** | 0.039392199367284775 |  0.03988051083352831 |  0.03648176168402036 |
| **best accuracy** | 0.8611111111111112 |  0.8472222222222222 |  0.8472222222222222 |
| **cloud accuracy** | 0.5 | 0.45 |  0.7 |
| **clear accuracy** | 1.0 |  1.0 |  0.9038461538461539 |

### Results
| Parameter   | Value       | Value       | Value       |
| ----------- | ----------- | ----------- | ----------- |
| Loss Func   | CrossEntropy | CrossEntropy | CrossEntropy |
| Kernel Size | 5 | 5 | 5 |
| Optimization | Adam | Adam | Adam |
| Learning Rate | 0.0001 | 0.0001 | 0.0001 |
| Epochs | 50 | 50 | 50 |
| Normalization | no | yes | yes |
| Bands used | 12 | 12 | 12 |
| Layers used | no Sigmoid | Sigmoid | Sigmoid |
| Augmentation | yes | yes | yes |
| Dropout | no | no | yes |
| **lowest loss** | 0.03655324007074038 | 0.5493429601192474 | 0.5491726431581709 |
| **best accuracy** | 0.8333333333333334 | 0.8472222222222222 | 0.8472222222222222 |
| **cloud accuracy** | 0.75 | 0.8 | 0.75 |
| **clear accuracy** | 0.8653846153846154 | 0.8653846153846154 | 0.8846153846153846 |

In [ ]:
### Results
| Parameter   | Value       | Value       | Value       |
| ----------- | ----------- | ----------- | ----------- |
| Loss Func   | CrossEntropy | CrossEntropy | CrossEntropy |
| Kernel Size | 5 | 5 | 5 |
| Optimization | Adam | Adam | Adam |
| Learning Rate | scheduler | scheduler | scheduler |
| Epochs | 50 | 50 | 50 |
| Normalization | yes | yes | yes |
| Bands used | 6 | 12 | 12 |
| Layers used | no Sigmoid | Sigmoid | Sigmoid |
| Augmentation | yes | yes | yes |
| Dropout | yes | no | yes |
| **lowest loss** |  |  |  |
| **best accuracy** |  | |  |
| **cloud accuracy** |  |  |  |
| **clear accuracy** |  |  |  |